## Docker images

In [1]:
run(pipeline(`docker images`, `grep openresty`, `grep -v 127.0.1.1`));

openresty                            bionic-with-top     a82cdeb5f7e6        36 hours ago        648MB
openresty/openresty                  bionic              de816c768659        2 weeks ago         562MB


In [2]:
;cat Dockerfile

# openresty image with ngxtop utility
#
# build using:
# docker build -t openresty:bionic-with-top -f Dockerfile .

FROM openresty/openresty:bionic

RUN apt-get install -y python-pip \
    && pip install ngxtop \
    && rm /usr/local/openresty/nginx/logs/access.log \
    && rm /usr/local/openresty/nginx/logs/error.log

COPY nginx.conf /usr/local/openresty/nginx/conf/nginx.conf


## Run a Nginx Webserver

In [3]:
using Kuber
ctx = KuberContext();

In [4]:
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


In [5]:
nginx_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "nginx-pod",
               "namespace": "default",
               "labels": {
                   "name": "nginx-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "nginx",
                   "image": "openresty:bionic-with-top",
                   "ports": [{"containerPort": 80}]
               }]
           }
       }""");

In [6]:
nginx_service = kuber_obj(ctx, """{
           "kind": "Service",
           "metadata": {
               "name": "nginx-service",
               "namespace": "default",
               "labels": {"name": "nginx-service"}
           },
           "spec": {
               "type": "NodePort",
               "ports": [{"port": 80}],
               "selector": {"name": "nginx-pod"}
           }
       }""");

In [7]:
typeof(nginx_pod), typeof(nginx_service)

(Kuber.Kubernetes.IoK8sApiCoreV1Pod, Kuber.Kubernetes.IoK8sApiCoreV1Service)

In [8]:
pod = put!(ctx, nginx_pod);

In [9]:
service = put!(ctx, nginx_service);

In [10]:
get(ctx, :Service, "nginx-service")

{
  "kind": "Service",
  "metadata": {
    "creationTimestamp": "2020-05-06T19:54:29Z",
    "uid": "b1a0134e-f621-4c3d-bf02-66e2ce803b9b",
    "labels": {
      "name": "nginx-service"
    },
    "selfLink": "/api/v1/namespaces/default/services/nginx-service",
    "name": "nginx-service",
    "managedFields": [
      {
        "fieldsType": "FieldsV1",
        "apiVersion": "v1",
        "manager": "HTTP.jl",
        "fieldsV1": {},
        "operation": "Update",
        "time": "2020-05-06T19:54:29Z"
      }
    ],
    "namespace": "default",
    "resourceVersion": "1858"
  },
  "apiVersion": "v1",
  "status": {
    "loadBalancer": {}
  },
  "spec": {
    "externalTrafficPolicy": "Cluster",
    "clusterIP": "10.101.116.169",
    "sessionAffinity": "None",
    "selector": {
      "name": "nginx-pod"
    },
    "ports": [
      {
        "nodePort": 32603,
        "port": 80,
        "protocol": "TCP",
        "targetPort": "80"
      }
    ],
    "type": "NodePort"
  }
}


In [11]:
;curl http://10.101.116.169/

<!DOCTYPE html>
<html>
<head>
<title>Welcome to OpenResty!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to OpenResty!</h1>
<p>If you see this page, the OpenResty web platform is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="https://openresty.org/">openresty.org</a>.<br/>
Commercial support is available at
<a href="https://openresty.com/">openresty.com</a>.</p>

<p><em>Thank you for flying OpenResty.</em></p>
</body>
</html>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   649  100   649    0     0   633k      0 --:--:-- --:--:-- --:--:--  633k


In [12]:
delete!(ctx, service);
delete!(ctx, pod);